In [1]:
# set parameters
 
file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

In [2]:
import pandas as pd
import numpy as np
import re

testdata = pd.read_csv(data_path+'full_test_samples.csv')
testdata

,prompts,answers,category
0,QUESTION787) Discuss the influence of the five...,E,psychology
1,QUESTION1172) A husband came home early from w...,A,law
2,QUESTION310) The Younger Dryas was:\n(A) a per...,C,history
3,QUESTION354) Homo habilis emerged around _____...,B,history
4,QUESTION2368) Mrs. Reynolds purchased $450 wor...,J,business
...,...,...,...
3605,QUESTION794) A buyer contracted in writing to ...,A,law
3606,QUESTION564) A sister and brother own a ranch ...,E,law
3607,QUESTION3445) For deception in an experiment t...,D,psychology
3608,QUESTION1995) A woman was married to a very we...,A,law


# ====================dot product

In [3]:
topics = ['ewha', 'philo', 'law', 'busin', 'psycho', 'his']
topics_embed = []

for idx, topic in enumerate(topics) :
    # .npy 파일 로드 (타입==넘파이)
    topicDB = np.load(data_path+f'topic_is/topic_is_{topic}.npy')
    topicDB_embed = np.load(data_path+f'topic_is/topic_is_{topic}_embed.npy')
    topicDB_embed = topicDB_embed.tolist()

    topics_embed.append(topicDB_embed)


len(topics_embed[0])

4096

In [4]:
from langchain_upstage import UpstageEmbeddings

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [5]:
which_category = pd.DataFrame(columns=['index', 'embed_ques', 'prompts', 'question', 'answers', 'category', 'pred_dot_category', 'pred_pmt_category', 'pred_category'])

for index, row in testdata.iterrows():
    #if index == 100 : break # 일단 실험할 땐 0개 단위로 끊어서 가져옴
    index = int(index)
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]

    try : 
        embedded_query = passage_embeddings.embed_query(question) # 질문 + 선지 받아와서 embedding 하기
        which_category.loc[len(which_category)] = {'index':index, 'embed_ques' : embedded_query, 'question': question, 'prompts' : q, 'answers' : a, 'category': row.category }
        # if re.search('[\uac00-\ud7a3]', question) != None :  # 한글 범위
        #     which_category.loc[index, 'pred_dot_category'] = 'ewha'

    except : # 여기서도 넣는 게 맞을 거 같음
        which_category.loc[len(which_category)] = {'index':index, 'embed_ques' : np.ones(4096), 'question': question, 'prompts' : q, 'answers' : a, 'category': row.category }
        print(f'pass: {index}')
        
    


pass: 3010


In [6]:
which_category

,index,embed_ques,prompts,question,answers,category,pred_dot_category,pred_pmt_category,pred_category
0,0,"[-0.00917816162109375, 0.0111083984375, 0.0121...",Discuss the influence of the five early school...,Discuss the influence of the five early school...,E,psychology,NaN,NaN,NaN
1,1,"[-0.00623321533203125, -0.00461578369140625, -...",A husband came home early from work one day. H...,A husband came home early from work one day. H...,A,law,NaN,NaN,NaN
2,2,"[0.0020771026611328125, -0.019927978515625, -0...",The Younger Dryas was:\n(A) a period of enviro...,The Younger Dryas was:\n,C,history,NaN,NaN,NaN
3,3,"[0.004932403564453125, -0.01318359375, -0.0223...",Homo habilis emerged around ________ years ago...,Homo habilis emerged around ________ years ago...,B,history,NaN,NaN,NaN
4,4,"[-0.0015153884887695312, 0.0152740478515625, -...",Mrs. Reynolds purchased $450 worth of carpetin...,Mrs. Reynolds purchased $450 worth of carpetin...,J,business,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3605,3605,"[-0.010498046875, -0.00653076171875, -0.026123...","A buyer contracted in writing to purchase 1,00...","A buyer contracted in writing to purchase 1,00...",A,law,NaN,NaN,NaN
3606,3606,"[-0.0062713623046875, 0.007228851318359375, -0...",A sister and brother own a ranch as joint tena...,A sister and brother own a ranch as joint tena...,E,law,NaN,NaN,NaN
3607,3607,"[0.0019121170043945312, 0.00170135498046875, -...",For deception in an experiment to be permissib...,For deception in an experiment to be permissib...,D,psychology,NaN,NaN,NaN
3608,3608,"[-0.0007233619689941406, -0.002759933471679687...",A woman was married to a very wealthy man who ...,A woman was married to a very wealthy man who ...,A,law,NaN,NaN,NaN


In [7]:
# Dot product

import numpy as np

for idx, row in which_category.iterrows() : # 질문 받아오기 
    if row.pred_dot_category == 'ewha' : continue
    
    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(topics_embed).T).argsort()[::-1]
    which_category.loc[idx, 'pred_dot_category'] = topics[sorted_idx[0]]
    

/tmp/ipykernel_430124/3690068492.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'psycho' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  which_category.loc[idx, 'pred_dot_category'] = topics[sorted_idx[0]]


In [8]:
# Cosine Sim
# '''
# import numpy as np
# import torch
# import torch.nn.functional as F

# for idx, row in which_category.iterrows() : # 질문 받아오기 

#     embed_ques= row.embed_ques

#         # 두 벡터 생성
#     ques = torch.tensor(embed_ques)
#     topi= torch.tensor(topics_embed)

#     # Cosine Similarity 계산
#     cos_sim = F.cosine_similarity(topi, ques.unsqueeze(0), dim=1)

#     # 유사도를 기준으로 내림차순 정렬
#     sorted_idx = torch.argsort(cos_sim, descending=True)  # 내림차순 정렬
#     which_category.loc[idx, 'pred_cos_category'] = topics[sorted_idx[0]]
# '''

In [9]:
#which_category[['prompts', 'category','pred_dot_category', 'pred_cos_category']].to_csv(data_path+'topic_is/topicistest_ver4.csv', sep=',')
which_category
# 오류 없이 full test samples / 선지 없이 발문만 주고 sim 계산  62.71 %
# full test samles / 선지까지 다 줬을 때 50.94%

,index,embed_ques,prompts,question,answers,category,pred_dot_category,pred_pmt_category,pred_category
0,0,"[-0.00917816162109375, 0.0111083984375, 0.0121...",Discuss the influence of the five early school...,Discuss the influence of the five early school...,E,psychology,psycho,NaN,NaN
1,1,"[-0.00623321533203125, -0.00461578369140625, -...",A husband came home early from work one day. H...,A husband came home early from work one day. H...,A,law,law,NaN,NaN
2,2,"[0.0020771026611328125, -0.019927978515625, -0...",The Younger Dryas was:\n(A) a period of enviro...,The Younger Dryas was:\n,C,history,his,NaN,NaN
3,3,"[0.004932403564453125, -0.01318359375, -0.0223...",Homo habilis emerged around ________ years ago...,Homo habilis emerged around ________ years ago...,B,history,his,NaN,NaN
4,4,"[-0.0015153884887695312, 0.0152740478515625, -...",Mrs. Reynolds purchased $450 worth of carpetin...,Mrs. Reynolds purchased $450 worth of carpetin...,J,business,ewha,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3605,3605,"[-0.010498046875, -0.00653076171875, -0.026123...","A buyer contracted in writing to purchase 1,00...","A buyer contracted in writing to purchase 1,00...",A,law,law,NaN,NaN
3606,3606,"[-0.0062713623046875, 0.007228851318359375, -0...",A sister and brother own a ranch as joint tena...,A sister and brother own a ranch as joint tena...,E,law,law,NaN,NaN
3607,3607,"[0.0019121170043945312, 0.00170135498046875, -...",For deception in an experiment to be permissib...,For deception in an experiment to be permissib...,D,psychology,law,NaN,NaN
3608,3608,"[-0.0007233619689941406, -0.002759933471679687...",A woman was married to a very wealthy man who ...,A woman was married to a very wealthy man who ...,A,law,law,NaN,NaN


In [16]:
for idx, row in which_category.iterrows() :
    if bool(re.search('[가-힣]', which_category.loc[idx, 'question'])) :
        #print('바ㅏ꿨어')
        which_category.loc[idx, 'pred_dot_category'] = 'ewha'

In [17]:
which_category.loc[len(which_category)-1]

index                                                             3609
embed_ques           [-0.0300750732421875, -0.0167388916015625, 0.0...
prompts              Identify the antecedent of the following condi...
question             Identify the antecedent of the following condi...
answers                                                              H
category                                                    philosophy
pred_dot_category                                                  law
pred_pmt_category                                                  NaN
pred_category                                                      NaN
Name: 3609, dtype: object

In [18]:
# 유사도 방법 두 개 비교

# cnt = 0
# wrong = []
# for i in range(len(which_category)) :

#     real = which_category['pred_dot_category'][i]
#     pred = which_category['pred_cos_category'][i]

#     if real != pred: print(i)



In [19]:
# 잘 구별했나 검사

cnt = 0
wrong_dot = []
for i in range(len(which_category)) :

    real = which_category['category'][i]
    pred = which_category['pred_dot_category'][i]

    #print(type(real))
    if real[:2] == pred[:2] : cnt +=1 
    else :
        wrong_dot.append(i)
        wrong_dot.append((real, pred))


acc = cnt/len(which_category)*100
print(f"acc: {acc}%")
#which_category.loc[len(which_category), 'predict'] = acc
#wrong

acc: 73.49030470914127%


In [20]:
from collections import Counter
count = Counter(wrong_dot)
filtered_counts = {item: cnt for item, cnt in count.items() if cnt > 1}
counts = sorted(filtered_counts.items(), key=lambda x: x[1], reverse=True)

counts

[(('business', 'ewha'), 241),
 (('psychology', 'law'), 107),
 (('philosophy', 'law'), 101),
 (('business', 'law'), 83),
 (('psychology', 'philo'), 75),
 (('history', 'law'), 59),
 (('psychology', 'his'), 50),
 (('philosophy', 'his'), 48),
 (('business', 'philo'), 28),
 (('business', 'his'), 26),
 (('psychology', 'busin'), 24),
 (('law', 'philo'), 20),
 (('psychology', 'ewha'), 18),
 (('business', 'psycho'), 17),
 (('history', 'philo'), 15),
 (('philosophy', 'busin'), 12),
 (('philosophy', 'psycho'), 7),
 (('philosophy', 'ewha'), 6),
 (('law', 'busin'), 5),
 (('law', 'ewha'), 4),
 (('history', 'psycho'), 4),
 (('law', 'his'), 3),
 (('history', 'busin'), 2),
 (('law', 'psycho'), 2)]

# =======================prompt

In [140]:
which_category

,index,embed_ques,prompts,question,answers,category,pred_dot_category,pred_pmt_category,pred_category
0,0,"[-0.00917816162109375, 0.0111083984375, 0.0121...",Discuss the influence of the five early school...,Discuss the influence of the five early school...,E,psychology,psycho,psychology,psychology
1,1,"[-0.00623321533203125, -0.00461578369140625, -...",A husband came home early from work one day. H...,A husband came home early from work one day. H...,A,law,law,law,law
2,2,"[0.0020771026611328125, -0.019927978515625, -0...",The Younger Dryas was:\n(A) a period of enviro...,The Younger Dryas was:\n,C,history,his,history,history
3,3,"[0.004913330078125, -0.01313018798828125, -0.0...",Homo habilis emerged around ________ years ago...,Homo habilis emerged around ________ years ago...,B,history,his,history,history
4,4,"[-0.0015153884887695312, 0.0152740478515625, -...",Mrs. Reynolds purchased $450 worth of carpetin...,Mrs. Reynolds purchased $450 worth of carpetin...,J,business,ewha,business,business
...,...,...,...,...,...,...,...,...,...
3605,3605,"[-0.010498046875, -0.00653076171875, -0.026123...","A buyer contracted in writing to purchase 1,00...","A buyer contracted in writing to purchase 1,00...",A,law,law,law,law
3606,3606,"[-0.0062713623046875, 0.007228851318359375, -0...",A sister and brother own a ranch as joint tena...,A sister and brother own a ranch as joint tena...,E,law,law,law,law
3607,3607,"[0.0019121170043945312, 0.00170135498046875, -...",For deception in an experiment to be permissib...,For deception in an experiment to be permissib...,D,psychology,law,philosophy,philosophy
3608,3608,"[-0.0007233619689941406, -0.002759933471679687...",A woman was married to a very wealthy man who ...,A woman was married to a very wealthy man who ...,A,law,law,law,law


In [152]:
# funcion to extract an answer from response

def extract_answer(idx, response):

    if any(keyword in response for keyword in ['philo', '철학']) : return 'philosophy'
    if any(keyword in response for keyword in ['law', '법']) : return 'law'
    if any(keyword in response for keyword in ['busin', '경영', '비즈니스', '경제', 'math', '수학']) : return 'business'
    if any(keyword in response for keyword in ['psycho', '심리', 'statistics','bio' ]) : return 'psychology'
    if any(keyword in response for keyword in ['his', '역사']) : return 'history'
    
    
    print(idx, '답없음', response , ' || 실제', which_category.loc[idx]['category'])
    print("==================")
    return 'philosophy'


In [ ]:
############# first PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are now assigned the role of classifying the topics of exam questions.

    Given the [Question] and [Topics], identify which topic the question belongs to.
    Do not provide any explanation, just return the topic name EXACTLY AS GIVEN.

    [Question] {question}
    [Topics] : (philosophy, law, business, psychology, history)

    Let's think step by step.
        
    '''

)
chain = prompt_template | llm


In [155]:
import numpy as np


for idx, row in which_category.iterrows() : # 질문 받아오기 
    #if idx == 3 : break

    response = chain.invoke({"question": row.question}) # top1에 대해서는 바로 답변 출력
    res = response.content
    res = extract_answer(idx, res.lower().replace("\n", " "))
    which_category.loc[idx, 'pred_pmt_category'] = res



115 답없음 the given question is related to the topic of [engineering].  || 실제 business
276 답없음 the question "objects that absorb light appear" belongs to the topic of [physics].  || 실제 psychology
608 답없음 교육학  || 실제 ewha
2063 답없음 economics  || 실제 business
2164 답없음 religion  || 실제 philosophy
2519 답없음 the topic of the question is [science].  || 실제 history
2716 답없음 medical  || 실제 psychology
2744 답없음 the given question does not fit into any of the provided topics. it seems to be a question about language or linguistics.  || 실제 psychology
3010 답없음 step 1:  examine the question and determine which topic it pertains to. step 2:  compare the question to the given list of topics. step 3:  identify the topic that the question is most closely related to. step 4:  return the topic name exactly as given.  answer: \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_  || 실제 business
3093 답없음 science  || 실제 psychology
3107 답없음 the topic of the given question is [science].  || 실제 business


In [156]:
for idx, res in enumerate(responses) :
    res = extract_answer(idx, res.lower())
    #print(res)
    which_category.loc[idx, 'pred_pmt_category'] = res

In [157]:
which_category.loc[1599]

index                                                             1599
embed_ques           [-0.01702880859375, -0.0175018310546875, -0.00...
prompts              Functionally, receptors in the retina of the e...
question             Functionally, receptors in the retina of the e...
answers                                                              J
category                                                    psychology
pred_dot_category                                               psycho
pred_pmt_category                                           psychology
pred_category                                               psychology
Name: 1599, dtype: object

In [158]:
which_category

,index,embed_ques,prompts,question,answers,category,pred_dot_category,pred_pmt_category,pred_category
0,0,"[-0.00917816162109375, 0.0111083984375, 0.0121...",Discuss the influence of the five early school...,Discuss the influence of the five early school...,E,psychology,psycho,psychology,psychology
1,1,"[-0.00623321533203125, -0.00461578369140625, -...",A husband came home early from work one day. H...,A husband came home early from work one day. H...,A,law,law,law,law
2,2,"[0.0020771026611328125, -0.019927978515625, -0...",The Younger Dryas was:\n(A) a period of enviro...,The Younger Dryas was:\n,C,history,his,history,history
3,3,"[0.004913330078125, -0.01313018798828125, -0.0...",Homo habilis emerged around ________ years ago...,Homo habilis emerged around ________ years ago...,B,history,his,history,history
4,4,"[-0.0015153884887695312, 0.0152740478515625, -...",Mrs. Reynolds purchased $450 worth of carpetin...,Mrs. Reynolds purchased $450 worth of carpetin...,J,business,ewha,business,business
...,...,...,...,...,...,...,...,...,...
3605,3605,"[-0.010498046875, -0.00653076171875, -0.026123...","A buyer contracted in writing to purchase 1,00...","A buyer contracted in writing to purchase 1,00...",A,law,law,law,law
3606,3606,"[-0.0062713623046875, 0.007228851318359375, -0...",A sister and brother own a ranch as joint tena...,A sister and brother own a ranch as joint tena...,E,law,law,law,law
3607,3607,"[0.0019121170043945312, 0.00170135498046875, -...",For deception in an experiment to be permissib...,For deception in an experiment to be permissib...,D,psychology,law,philosophy,psychology
3608,3608,"[-0.0007233619689941406, -0.002759933471679687...",A woman was married to a very wealthy man who ...,A woman was married to a very wealthy man who ...,A,law,law,law,law


In [159]:
# 잘 구별했나 검사

cnt = 0
wrong_pmt = []
try : 
    for i in range(len(which_category)) :
        
    # if i == 1000 : break # 실험
        
        real = which_category['category'][i]
        pred = which_category['pred_pmt_category'][i]

        #print(type(real))
        if real[:2] == pred[:2] : cnt +=1 

        else :
            wrong_pmt.append(i)
            wrong_pmt.append((real, pred))

except : pass

acc = cnt/len(which_category)*100
print(f"acc: {acc}%")
#which_category.loc[len(which_category), 'predict'] = acc
#wrong


acc: 88.14404432132964%


In [160]:
from collections import Counter
count = Counter(wrong_pmt)
filtered_counts = {item: cnt for item, cnt in count.items() if cnt > 1}
counts = sorted(filtered_counts.items(), key=lambda x: x[1], reverse=True)

counts

[(('psychology', 'philosophy'), 81),
 (('philosophy', 'history'), 43),
 (('business', 'philosophy'), 41),
 (('ewha', 'law'), 34),
 (('law', 'philosophy'), 33),
 (('business', 'psychology'), 30),
 (('psychology', 'business'), 29),
 (('history', 'philosophy'), 29),
 (('business', 'law'), 20),
 (('philosophy', 'law'), 17),
 (('psychology', 'law'), 17),
 (('law', 'business'), 9),
 (('philosophy', 'psychology'), 7),
 (('history', 'psychology'), 7),
 (('history', 'law'), 7),
 (('law', 'history'), 4),
 (('ewha', 'philosophy'), 4),
 (('psychology', 'history'), 4),
 (('philosophy', 'business'), 3),
 (('business', 'history'), 3),
 (('ewha', 'history'), 2),
 (('history', 'business'), 2)]

프롬프트로 하ㅕㄴ 73정도 나옴
-> 답변 키워드만 내라는데 자꾸 주절주절 거려서 그런 듯
토픽을 뽑아내는 코드를 적어야 하나 
그리고 답변에 없으면 유사도 기반으로 ....

# 최종 카테고리 선정

In [161]:
which_category['pred_category'] = which_category['pred_pmt_category']

# which_category.loc[which_category['pred_dot_category'] == 'ewha', 'pred_category'] = 'ewha'

for idx, row in which_category.iterrows() :
    if bool(re.search('[가-힣]', which_category.loc[idx, 'question'])) :
        #print('바ㅏ꿨어')
        which_category.loc[idx, 'pred_category'] = 'ewha'


which_category

,index,embed_ques,prompts,question,answers,category,pred_dot_category,pred_pmt_category,pred_category
0,0,"[-0.00917816162109375, 0.0111083984375, 0.0121...",Discuss the influence of the five early school...,Discuss the influence of the five early school...,E,psychology,psycho,psychology,psychology
1,1,"[-0.00623321533203125, -0.00461578369140625, -...",A husband came home early from work one day. H...,A husband came home early from work one day. H...,A,law,law,law,law
2,2,"[0.0020771026611328125, -0.019927978515625, -0...",The Younger Dryas was:\n(A) a period of enviro...,The Younger Dryas was:\n,C,history,his,history,history
3,3,"[0.004913330078125, -0.01313018798828125, -0.0...",Homo habilis emerged around ________ years ago...,Homo habilis emerged around ________ years ago...,B,history,his,history,history
4,4,"[-0.0015153884887695312, 0.0152740478515625, -...",Mrs. Reynolds purchased $450 worth of carpetin...,Mrs. Reynolds purchased $450 worth of carpetin...,J,business,ewha,business,business
...,...,...,...,...,...,...,...,...,...
3605,3605,"[-0.010498046875, -0.00653076171875, -0.026123...","A buyer contracted in writing to purchase 1,00...","A buyer contracted in writing to purchase 1,00...",A,law,law,law,law
3606,3606,"[-0.0062713623046875, 0.007228851318359375, -0...",A sister and brother own a ranch as joint tena...,A sister and brother own a ranch as joint tena...,E,law,law,law,law
3607,3607,"[0.0019121170043945312, 0.00170135498046875, -...",For deception in an experiment to be permissib...,For deception in an experiment to be permissib...,D,psychology,law,philosophy,philosophy
3608,3608,"[-0.0007233619689941406, -0.002759933471679687...",A woman was married to a very wealthy man who ...,A woman was married to a very wealthy man who ...,A,law,law,law,law


In [162]:
# 잘 구별했나 검사

cnt = 0
wrong = []
try : 
    for i in range(len(which_category)) :
        
    # if i == 1000 : break # 실험
        
        real = which_category['category'][i]
        pred = which_category['pred_category'][i]

        #print(type(real))
        if real[:2] == pred[:2] : cnt +=1 

        else :
            wrong.append(i)
            wrong.append((real, pred))

except : pass

acc = cnt/len(which_category)*100
print(f"acc: {acc}%")
#which_category.loc[len(which_category), 'predict'] = acc
#wrong


acc: 89.30747922437673%


In [163]:
from collections import Counter
count = Counter(wrong)
filtered_counts = {item: cnt for item, cnt in count.items() if cnt > 1}
counts = sorted(filtered_counts.items(), key=lambda x: x[1], reverse=True)

counts

[(('psychology', 'philosophy'), 81),
 (('philosophy', 'history'), 43),
 (('business', 'philosophy'), 41),
 (('law', 'philosophy'), 33),
 (('business', 'psychology'), 30),
 (('psychology', 'business'), 29),
 (('history', 'philosophy'), 29),
 (('business', 'law'), 20),
 (('philosophy', 'law'), 17),
 (('psychology', 'law'), 17),
 (('law', 'business'), 9),
 (('philosophy', 'psychology'), 7),
 (('history', 'psychology'), 7),
 (('history', 'law'), 7),
 (('law', 'history'), 4),
 (('psychology', 'history'), 4),
 (('philosophy', 'business'), 3),
 (('business', 'history'), 3),
 (('history', 'business'), 2)]